### 1. Cargar y preparar los datos

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



# Cargar datos
data = pd.read_csv('/Users/juanvielmapereyra/NOTEBOOKS/SPRINT 10/SPRINT_10/Churn.csv')

# Rellenar valores faltantes
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())

# Eliminar columnas innecesarias
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Codificar variables categóricas
data = pd.get_dummies(data, drop_first=True)

# Separar objetivo y características
target = data['Exited']
features = data.drop('Exited', axis=1)

# Dividir datos en entrenamiento y prueba
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

# Estandarizar características numéricas
scaler = StandardScaler()
features_train = pd.DataFrame(scaler.fit_transform(features_train), index=features_train.index, columns=features.columns)
features_test = pd.DataFrame(scaler.transform(features_test), index=features_test.index, columns=features.columns)


### Procedimiento de preparación:

Eliminar columnas que no aportan valor al modelo (RowNumber, CustomerId, Surname).

Codificar variables categóricas (Geography, Gender → usar OHE para Geography y 0/1 para Gender si es posible).

Estandarizar las características numéricas.

In [22]:
!pip install scikit-learn



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### EQUILIBRIO DE CLASES

In [23]:
print(target_train.value_counts(normalize=True))


Exited
0    0.799733
1    0.200267
Name: proportion, dtype: float64


### MODELO BASE SIN CORREGIR EL DESEQUILIBRIO

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score

model_base = RandomForestClassifier(random_state=12345)
model_base.fit(features_train, target_train)

predicted_test = model_base.predict(features_test)
probabilities_test = model_base.predict_proba(features_test)[:, 1]

print("F1 base:", f1_score(target_test, predicted_test))
print("AUC-ROC base:", roc_auc_score(target_test, probabilities_test))


F1 base: 0.5744431418522861
AUC-ROC base: 0.8524001807329197


### CORRECCIÒN DEL DESEQUILIBRIO (UPSAMPLING)

In [25]:
from sklearn.utils import shuffle

def upsample(features, target, repeat=5):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_ones_upsampled = pd.concat([features_ones] * repeat)
    target_ones_upsampled = pd.concat([target_ones] * repeat)

    features_upsampled = pd.concat([features_zeros, features_ones_upsampled])
    target_upsampled = pd.concat([target_zeros, target_ones_upsampled])

    return shuffle(features_upsampled, target_upsampled, random_state=12345)

features_train_up, target_train_up = upsample(features_train, target_train)

model_up = RandomForestClassifier(random_state=12345)
model_up.fit(features_train_up, target_train_up)

pred_up = model_up.predict(features_test)
proba_up = model_up.predict_proba(features_test)[:, 1]

print("F1 con upsampling:", f1_score(target_test, pred_up))
print("AUC-ROC con upsampling:", roc_auc_score(target_test, proba_up))


F1 con upsampling: 0.6134715025906736
AUC-ROC con upsampling: 0.8529756723978027


### MÈTODO 2: class_weight='balanced'

In [26]:
model_weighted = RandomForestClassifier(class_weight='balanced', random_state=12345)
model_weighted.fit(features_train, target_train)

pred_weighted = model_weighted.predict(features_test)
proba_weighted = model_weighted.predict_proba(features_test)[:, 1]

print("F1 con ponderación:", f1_score(target_test, pred_weighted))
print("AUC-ROC con ponderación:", roc_auc_score(target_test, proba_weighted))


F1 con ponderación: 0.5641646489104116
AUC-ROC con ponderación: 0.8532648450690828


### MEJOR MODELO

In [27]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15],
}

grid = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=12345),
                    param_grid, scoring='f1', cv=3)

grid.fit(features_train, target_train)

print("Mejores parámetros:", grid.best_params_)


Mejores parámetros: {'max_depth': 10, 'n_estimators': 100}


### PRUEBA DEFINITIVA

In [28]:
final_model = grid.best_estimator_  # o model_weighted, o model_up
pred_final = final_model.predict(features_test)
proba_final = final_model.predict_proba(features_test)[:, 1]

print("F1 final:", f1_score(target_test, pred_final))
print("AUC-ROC final:", roc_auc_score(target_test, proba_final))


F1 final: 0.64376130198915
AUC-ROC final: 0.8654728781717438


### COMPARACION FINAL

In [29]:
print("\nReporte final (mejor modelo):")
print(classification_report(target_test, pred_weighted))
print("\nMatriz de confusión:")
print(confusion_matrix(target_test, pred_weighted))


Reporte final (mejor modelo):
              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1965
           1       0.80      0.44      0.56       535

    accuracy                           0.86      2500
   macro avg       0.83      0.70      0.74      2500
weighted avg       0.85      0.86      0.84      2500


Matriz de confusión:
[[1907   58]
 [ 302  233]]


## CONCLUSION FINAL DEL PROYECTO
#### El modelo final, entrenado con corrección de desequilibrio mediante ponderación de clases (class_weight='balanced'), logró un rendimiento global sólido con una precisión general del 86% y un F1-score ponderado de 0.84.

#### Sin embargo, el F1-score para la clase positiva (clientes que abandonan) fue de 0.56, quedando justo por debajo del umbral mínimo de 0.59 requerido. El modelo detecta el 44% de los clientes que realmente abandonan (recall) y acierta en el 80% de los casos cuando predice que alguien se va (precisión).

#### La matriz de confusión muestra que 302 clientes que se fueron no fueron detectados (falsos negativos), mientras que 58 fueron clasificados erróneamente como salientes (falsos positivos).